In [1]:
import tensorflow as tf
# from utils import get_data_info, read_data, load_word_embeddings
from IAN_model import IAN
import pandas as pd
import numpy as np
import os
import time

/usr/local/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [30]:
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# FLAGS = tf.app.flags.FLAGS
# tf.app.flags.DEFINE_integer('embedding_dim', 64, 'dimension of word embedding')
# tf.app.flags.DEFINE_integer('batch_size', 128, 'number of example per batch')
# tf.app.flags.DEFINE_integer('n_epoch', 10, 'number of epoch')
# tf.app.flags.DEFINE_integer('n_hidden', 300, 'number of hidden unit')
# tf.app.flags.DEFINE_integer('n_class', 3, 'number of distinct class')
# tf.app.flags.DEFINE_integer('pre_processed', 0, 'Whether the data is pre-processed')
# tf.app.flags.DEFINE_float('learning_rate', 0.01, 'learning rate')
# tf.app.flags.DEFINE_float('l2_reg', 0.001, 'l2 regularization')
# tf.app.flags.DEFINE_float('dropout', 0.5, 'dropout')


# tf.app.flags.DEFINE_integer('max_aspect_len', 1, 'max length of aspects')
# tf.app.flags.DEFINE_integer('max_context_len', 90, 'max length of contexts')
# tf.app.flags.DEFINE_string('embedding_matrix', '', 'word ids to word vectors')

ArgumentError: argument --embedding_dim: conflicting option string: --embedding_dim

In [78]:
config = {
    'embedding_dim':64,
    'n_class': 4,
    'batch_size': 1024,
    'pre_processed': 0,
    'max_aspect_len': 1,
    'max_context_len': 90, 

    'n_epoch': 5,
    'n_hidden': 100,
    'learning_rate': 0.01,
    'l2_reg': 0.001,   
    'dropout': 0.5,
}

In [14]:
# train.head()

,content_id,content,subject,sentiment_value,sentiment_word
0,vUXizsqexyZVRdFH,因为森林人即将换代，这套系统没必要装在一款即将换代的车型上，因为肯定会影响价格。,价格,0,影响
1,4QroPd9hNfnCHVt7,四驱价格貌似挺高的，高的可以看齐XC60了，看实车前脸有点违和感。不过大众的车应该不会差。,价格,-1,高
2,QmqJ2AvM5GplaRyz,斯柯达要说质量，似乎比大众要好一点，价格也低一些，用料完全一样。我听说过野帝，但没听说过你说...,价格,1,低
3,KMT1gFJiU4NWrVDn,这玩意都是给有钱任性又不懂车的土豪用的，这价格换一次我妹夫EP020可以换三锅了,价格,-1,有钱任性
4,nVIlGd5yMmc37t1o,17价格忒高，估计也就是14-15左右。,价格,-1,高


In [196]:
def get_stop_word_set():
    words_set = set()
    with open('/home/cjy/extract_tag/data/哈工大停用词表扩展.txt') as f_r:
        for line in f_r:
            words_set |= set(line.strip())
#     print('stop words cnt:', len(words_set))
    return words_set

In [209]:
def get_word2id(data_path, all_subjects, pre_processed, train_fname, test_fname):
    '''构造 word id 映射'''
    save_fname = data_path + 'word2id_map_without_stopword.txt'

    if pre_processed:
        pass
#         TODO
    else:
        import jieba
        jieba.load_userdict('../data/cars.txt')
        stop_words = get_stop_word_set()
        word2id = {}
        word2id['<pad>'] = 0
        for s in all_subjects:
            word2id[s] = len(word2id)
        train_fname = data_path + train_fname + '.csv'
        test_fname = data_path + test_fname + '.csv'
        
        data = pd.read_csv(train_fname)
        data = pd.concat([data, pd.read_csv(test_fname)])
        data = data.drop_duplicates('content_id')
        for content in data['content'].values:
            content = content.strip()
            for word in jieba.cut(content):
                if word not in word2id and word not in stop_words:
                    word2id[word] = len(word2id)

        with open(save_fname, 'w') as fsave:
            for item in sorted(word2id.items(), key=lambda x:x[1]):
                fsave.write(item[0]+' '+str(item[1])+'\n')

    return word2id

subjects = ['动力', '价格', '内饰', '配置', '安全性', '外观', '操控', '油耗', '空间', '舒适性']
word2id = get_word2id('../data/', subjects, False, 'train', 'test_public')
len(word2id)

19324

In [210]:
def read_train_data(word2id, max_context_len, data_path, fname, all_aspect, pre_processed):
    '''return aspects, contents, labels, aspect_lens, content_lens'''
    if pre_processed:
        pass
    else:
        import jieba
        jieba.load_userdict('../data/cars.txt')
        stop_words = get_stop_word_set()
        aspects, contents, labels, content_lens = [], [], [], []
        data = pd.read_csv(data_path + fname + '.csv')
        id2label = data.groupby('content_id').apply(lambda x:dict([(item[0], item[1]) for item in x[['subject', 'sentiment_value']].values])).to_dict()
        data = data.drop_duplicates('content_id')
        data.reset_index(drop=True, inplace=True)
        f_w = open(data_path + fname + '_info_without_stopword.txt', 'w')
        for item in data[['content_id', 'content']].values:
#             print(item[0])
            words = [word2id[x] for x in filter(lambda x:x not in stop_words, jieba.cut(item[1].strip()))]
            f_w.write(str(words) + '\n')
            if len(words) < max_context_len:
                crt_content_lens = len(words)
                words = words + [0] * (max_context_len-len(words))
            else:
                crt_content_lens = max_context_len
                words = words[:max_context_len]
            
            crt_subject_value = id2label[item[0]]
            f_w.write(str(crt_subject_value) + '\n')
            for suj in all_aspect:
                if suj in crt_subject_value:
                    contents.append(words)
                    aspects.append([word2id[suj]])
                    content_lens.append(crt_content_lens)
                    crt_label = [0] * 4
                    crt_label[crt_subject_value[suj]+1] = 1
                    labels.append(crt_label)
                else:
                    contents.append(words)
                    aspects.append([word2id[suj]])
                    content_lens.append(crt_content_lens)
                    labels.append([0] * 3 + [1])
    
    return np.asarray(aspects), np.asarray(contents), np.asarray(labels), np.ones(len(aspects)), np.asarray(content_lens)
            
train = read_train_data(word2id, config['max_context_len'], '../data/', 'train',subjects, False)
print(len(train[0]))

82900


In [230]:
def read_test_data(word2id, max_context_len, data_path, fname, all_aspect, pre_processed):
    '''return aspects, contents, aspect_lens, content_lens'''
    if pre_processed:
        pass
    else:
        import jieba
        jieba.load_userdict('../data/cars.txt')
        stop_words = get_stop_word_set()
        aspects, contents, content_lens = [], [], []
        data = pd.read_csv(data_path + fname + '.csv')
        data = data.drop_duplicates('content_id')
        data.reset_index(drop=True, inplace=True)
        f_w = open(data_path + fname + '_info_without_stopword.txt', 'w')
        for item in data[['content_id', 'content']].values:
            words = [word2id[x] for x in filter(lambda x:x not in stop_words, jieba.cut(item[1].strip()))]
            f_w.write(str(words) + '\n')
            if len(words) < max_context_len:
                crt_content_lens = len(words)
                words = words + [0] * (max_context_len-len(words))
            else:
                crt_content_lens = max_context_len
                words = words[:max_context_len]
            
            for suj in all_aspect:
                contents.append(words)
                aspects.append([word2id[suj]])
                content_lens.append(crt_content_lens)
    
    return np.asarray(aspects), np.asarray(contents), np.ones(len(aspects)), np.asarray(content_lens)
test = read_test_data(word2id, config['max_context_len'], '../data/', 'test_public',subjects, False)
print(len(test[0]))

23640


In [211]:
def load_word_embeddings(word2id):
    from gensim.models import KeyedVectors
    com_w2v = KeyedVectors.load_word2vec_format('/home/cjy/extract_tag/model/news_12g_baidubaike_20g_novel_90g_embedding_64.bin', 
                                                binary=True)
    com_w2v_embedding_dim = 64
    
    word2vec = np.random.uniform(-0.01, 0.01, [len(word2id), com_w2v_embedding_dim])
    for w, w_id in word2id.items():
        if w in com_w2v:
            word2vec[word2id[w]] = com_w2v[w]
    word2vec[word2id['<pad>'], :] = 0
    return word2vec
    
config['embedding_matrix'] = load_word_embeddings(word2id)

In [252]:
config['early_stop'] = 1
config['n_epoch'] = 100
config['learning_rate'] = 0.01
config['n_hidden'] = 300

In [ ]:
from imp import reload
import IAN_model
reload(IAN_model)
from IAN_model import IAN

sess.close()

tf.reset_default_graph()
sess = tf.Session();

model = IAN(config, sess)
model.build_model()
train_cnt = int(len(train[0]) * 0.7)
model.run((train[0][:train_cnt], train[1][:train_cnt], train[2][:train_cnt], train[3][:train_cnt], train[4][:train_cnt]),
          (train[0][train_cnt:], train[1][train_cnt:], train[2][train_cnt:], train[3][train_cnt:], train[4][train_cnt:]))

Training ...


In [243]:
preds = model.predict_data(test)

In [251]:
# preds = np.asarray(preds)
# max_idx = np.argmax(preds, axis=1)
# max_idx_score = np.max(preds, axis=1)



,a
0,"[1, 2]"
1,"[2, 3]"


In [180]:
# train[2].sum(axis=0)

array([ 1616,  6661,  1670, 72953])